In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import modules

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
sample_image = tf.keras.preprocessing.image.load_img('/kaggle/input/animals/animals/train/cat/cat1.jpg', target_size= (224, 224))

In [ ]:
plt.imshow(sample_image)

## Create Train and Validation Generator datas

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input)

In [ ]:
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input)

In [ ]:
train_generator = train_datagen.flow_from_directory("/kaggle/input/animals/animals/train",
                                                   target_size = (224 , 224),
                                                   color_mode = 'rgb',
                                                   batch_size = 32,
                                                   class_mode = 'categorical',
                                                   shuffle = True)

In [ ]:
val_generator = val_datagen.flow_from_directory("/kaggle/input/animals/animals/val",
                                                   target_size = (224 , 224),
                                                   color_mode = 'rgb',
                                                   batch_size = 32,
                                                   class_mode = 'categorical',
                                                   shuffle = True)

# Create a model

In [ ]:
base_model = tf.keras.applications.ResNet50(weights = 'imagenet', include_top = False)

In [ ]:
base_model.summary()

In [ ]:
x = base_model.output

In [ ]:
x = tf.keras.layers.GlobalAveragePooling2D()(x)

In [ ]:
x = tf.keras.layers.Dense(1024, activation= 'relu')(x)
x = tf.keras.layers.Dense(1024, activation= 'relu')(x)
x = tf.keras.layers.Dense(1024, activation= 'relu')(x)
x = tf.keras.layers.Dense(512, activation= 'relu')(x)
preds = tf.keras.layers.Dense(5, activation= 'softmax')(x)

In [ ]:
model = tf.keras.models.Model(inputs = base_model.input, outputs = preds)

In [ ]:
model.summary()

# freezing layers

In [ ]:
for i, layer in enumerate(model.layers):
    print(i, layer.name)

In [ ]:
for layer in model.layers[:175]:
    layer.treinable = False

In [ ]:
for layer in model.layers[175:]:
    layer.treinable = True

# Compile and Train model

In [ ]:
model.compile(optimizer = 'Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, epochs=25)

# Historical loss and accuracy

In [ ]:
acc = history.history['accuracy']
loss = history.history['loss']

plt.figure()
plt.plot(acc, label='Training Accuracy')
plt.ylabel('Accuracy')
plt.title('Training Accuracy')

plt.figure()

plt.plot(loss, label = 'Training loss')
plt.ylabel('loss')
plt.title('Training loss')
plt.show()

# Tests on validation data

## Test on single sample image

In [ ]:
sample_image2 = tf.keras.preprocessing.image.load_img('/kaggle/input/animals/animals/val/horse/horse1.png', target_size = (224, 224))

In [ ]:
class_names = val_generator.class_indices
# class_names = sorted(val_generator.class_indices.items(), key = lambda pair:pair[1])
# class_names = np.array([key.title() for key, value in class_names])

In [ ]:
class_names

In [ ]:
plt.imshow(sample_image2)

In [ ]:
sample_image2 = tf.keras.preprocessing.image.img_to_array(sample_image2)

In [ ]:
sample_image2 = np.expand_dims(sample_image2, axis = 0)

In [ ]:
sample_image2 = tf.keras.applications.resnet50.preprocess_input(sample_image2)

In [ ]:
predictions = model.predict(sample_image2).argmax()
print(predictions)

In [ ]:
# class_names[predictions]

In [ ]:
class_name = list({k for k in class_names if class_names[k]==predictions})[0]

In [ ]:
class_name

## Test validation data

In [ ]:
predictions = model.predict(val_generator).argmax(axis=1)

In [ ]:
predictions.shape

In [ ]:
predictions

In [ ]:
y_true = val_generator.classes

In [ ]:
predicted_names = []
for prediction in predictions:
    predicted_names.append({k for k in class_names if class_names[k]==predictions[prediction]})

In [ ]:
import plotly.express as px

fig = px.density_heatmap(x=predictions, y=y_true, text_auto=True).update_layout(
    xaxis_title="Predicted Label", yaxis_title="True Label")
fig.show()

In [ ]:
# predicted_names

In [ ]:
model.save_weights("weights_animals_resnet.h5")